In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import json
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from flask import Flask,send_file
from flask import jsonify
from flask_cors import CORS
from flask import Response
from sklearn.metrics import mean_squared_error
import base64
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
plt.style.use('seaborn')
import seaborn as sns
import numpy as np
from io import StringIO 
import pandas as pd
%matplotlib inline
import base64
import json 
from json import dumps
from flask import request
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import statsmodels.api as sm
import io
import random
app = Flask(__name__)

CORS(app)
@app.route('/image')
def serve_image():
    CleanDataset=pd.read_csv('DailyDataset.csv')
    columns = CleanDataset.iloc[:, 1:9]
    sizes = columns.sum(axis=0)
    plt.ion()
    plt.clf()

    plt.figure(figsize=(8, 5))
    sizes.plot.pie()
    buf = io.BytesIO()
    
    # Save the figure to the BytesIO object
    plt.savefig(buf, format='png')

    # Seek to the beginning of the BytesIO object
    buf.seek(0)

    # Create a Flask Response object
    response = Response(buf.getvalue(), mimetype='image/png')
    plt.ioff()
    # Send the Response object to the client
    return response

def Dailydata(weekly):
    weekly["date"] = pd.to_datetime(weekly["date"])
    weekly_summary =weekly.resample('w' ,on='date').agg({ 'M01AB' :'sum', 'M01AE' : 'sum', 'N02BA' :'sum', 'N02BE' : 'sum', 'N05B' :'sum', 'N05C' : 'sum', 'R03' :'sum','R06' :'sum', 'Temperature' :'mean', 'Dew Point' :'mean', 'Humidity' :'mean',  'Wind Speed' :'mean', 'Pressure' :'mean',   'IntCondition' : 'mean'})
    weekly_summary['date']=weekly_summary.index
    return weekly_summary
def Weeklydata(weekly):
    weekly["date"] = pd.to_datetime(weekly["date"])
    weekly_summary =weekly.resample('w' ,on='date').agg({ 'M01AB' :'sum', 'M01AE' : 'sum', 'N02BA' :'sum', 'N02BE' : 'sum', 'N05B' :'sum', 'N05C' : 'sum', 'R03' :'sum','R06' :'sum', 'Temperature' :'mean', 'Dew Point' :'mean', 'Humidity' :'mean',  'Wind Speed' :'mean', 'Pressure' :'mean',   'IntCondition' : 'mean'})
    weekly_summary['date']=weekly_summary.index
    return weekly_summary
def Monthlydata(weekly):
    monthly = pd.DataFrame()

    weekly["date"] = pd.to_datetime(weekly["date"])
    Monthly_summary =weekly.resample('M' ,on='date').agg({ 'M01AB' :'sum', 'M01AE' : 'sum', 'N02BA' :'sum', 'N02BE' : 'sum', 'N05B' :'sum', 'N05C' : 'sum', 'R03' :'sum','R06' :'sum', 'Temperature' :'mean', 'Dew Point' :'mean', 'Humidity' :'mean',  'Wind Speed' :'mean', 'Pressure' :'mean',   'IntCondition' : 'mean'})
    Monthly_summary['date']=Monthly_summary.index
    return Monthly_summary
def Yearlydata(weekly):
    weekly["date"] = pd.to_datetime(weekly["date"])
    yearly_summary =weekly.resample('Y' ,on='date').agg({ 'M01AB' :'sum', 'M01AE' : 'sum', 'N02BA' :'sum', 'N02BE' : 'sum', 'N05B' :'sum', 'N05C' : 'sum', 'R03' :'sum','R06' :'sum', 'Temperature' :'mean', 'Dew Point' :'mean', 'Humidity' :'mean',  'Wind Speed' :'mean', 'Pressure' :'mean',   'IntCondition' :'mean'})
    yearly_summary['date']=yearly_summary.index
    return yearly_summary

@app.route('/GetGraphs',methods=['GET', 'POST'])
def dashboardgraphs():
    dailydataset=pd.read_csv('DailyDataset.csv')
    if request.method == 'POST':
        content = request.json
        startDate=content["startDate"]
        endDate=content["endDate"]
        types=content["type"]
         #3 attributes startdate enddate datetype
        if(types=='Weekly'):
             dailydataset=Dailydata(dailydataset)
        elif(types=='Monthly'):
             dailydataset=Monthlydata(dailydataset)
        else:
             dailydataset=Yearlydata(dailydataset)
        plotgraph(dailydataset,startDate,endDate)
        with open("plot.png", "rb") as img_file:
            my_img = base64.b64encode(img_file.read())
            base64_string = my_img .decode('utf-8')
    else:
        dailydataset=Dailydata(CleanDataset)
        plotgraph(dailydataset,'2017-06-01','2017-07-13')
        with open("plot.png", "rb") as img_file:
            my_img = base64.b64encode(img_file.read())
            base64_string = my_img .decode('utf-8')
    return jsonify(base64_string)


@app.route('/data', methods=['GET', 'POST'])
def data():
    z=process_data()
    print(z)
    return z
    

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

    
def plotgraph(dataset,start_date,end_date):
    plt.ion()
    plt.clf()

    dataset['date'] = pd.to_datetime(dataset['date'])
    df2 = dataset.query('date>= @start_date and date<=@end_date')
    Date = df2['date']
    M01AB = df2['M01AB']
    M01AE = df2['M01AE']
    N02BA = df2['N02BA']
    N02BE = df2['N02BE']
    N05B = df2['N05B']
    N05C = df2['N05C']
    R03 = df2['R03']
    R06 = df2['R06']
    plt.figure(figsize=(8, 5))
    plt.rcParams['figure.figsize'] = (14,9)
    plt.rcParams['date.converter'] = 'concise'
    plt.rcParams['date.interval_multiples'] = True
    plt.plot(Date, M01AB, color='tab:purple', label='M01AB')
    plt.plot(Date, M01AE, color='tab:blue', label='M01AE')
    plt.plot(Date, N02BA, color='tab:pink', label='N02BA')
    plt.plot(Date, N02BE, color='tab:red', label='N02BE')
    plt.plot(Date, N05B, color='tab:brown', label='N05BE')
    plt.plot(Date, N05C, color='tab:gray', label='N05C')
    plt.plot(Date, R03, color='tab:cyan', label='R03')
    plt.plot(Date, R06, color='tab:olive', label='R06')
    plt.legend()
    plt.title('Sales ')
    plt.xlabel('Date')
    plt.ylabel('Sales numbers')
    plt.xticks(rotation=45)
    plt.savefig('plot.png')
    plt.ioff()

@app.route('/Applygradientdecent',methods=['GET', 'POST'])    
def PredictValueAccordingTomodel():
    
    if request.method == 'POST':
        
        CleanDataset=pd.read_csv('CleanDataset.csv')
        content = request.json
        medname=content["MedName"]
        types=content["typeselect"]
        Temperature= content['Temperature'] 
        Dew_Point= content['Dew_Point']
        Wind_Speed= content['Wind_Speed'] 
        Humidity= content['Humidity']
        Pressure= content['Pressure']
        Condition_int=content['Condition_int']
        if(types=='Weekly'):
            dailydataset=Dailydata(CleanDataset)
        elif(types=='Monthly'):
            dailydataset=Monthlydata(CleanDataset)
        else:
            dailydataset=Yearlydata(CleanDataset)
        Value=eq2(medname,Temperature, Dew_Point, Wind_Speed, Humidity, Pressure,Condition_int)
        print(Value)
        my_json = json.dumps({'PredictedValues': Value}, cls=NpEncoder)
        return my_json
    



if __name__=="__main__":
    app.run(host='0.0.0.0', port=105)
    

C:\Users\Ibrahim\AppData\Local\Temp\ipykernel_6200\2066540876.py:22: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:105
 * Running on http://192.168.43.236:105
Press CTRL+C to quit
127.0.0.1 - - [05/Jun/2023 19:55:51] "GET /image HTTP/1.1" 200 -
192.168.43.236 - - [05/Jun/2023 19:56:34] "POST /GetGraphs HTTP/1.1" 200 -
192.168.43.236 - - [05/Jun/2023 19:56:50] "POST /GetGraphs HTTP/1.1" 200 -
192.168.43.236 - - [05/Jun/2023 19:57:02] "POST /GetGraphs HTTP/1.1" 200 -


Testing Linear Regression...


C:\Users\Ibrahim\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


...

Best parameters: {'alpha': 40}
Training score: 0.6070367560592033
Testing score: 0.6304955308087963
Mean Squared Error (MSE): 8.58679008619985
Root Mean Squared Error (RMSE): 2.930322522556152
R-squared (R2): 0.6304955308087963
Mean Absolute Error (MAE): 2.335536345699809
Testing Decision Tree...
Best parameters: {'max_depth': 5}
Training score: 0.8021230072458867
Testing score: 0.81511778435401
Mean Squared Error (MSE): 4.296415629014105
Root Mean Squared Error (RMSE): 2.072779686559598
R-squared (R2): 0.81511778435401
Mean Absolute Error (MAE): 1.581181404949827
Testing Random Forest...


C:\Users\Ibrahim\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters: {'n_estimators': 50, 'max_depth': 5}
Training score: 0.8102163258867436
Testing score: 0.8286586779764444
Mean Squared Error (MSE): 3.981743356254019
Root Mean Squared Error (RMSE): 1.9954306192534028
R-squared (R2): 0.8286586779764444
Mean Absolute Error (MAE): 1.5379468251913262
Testing Gradient Boosting...


C:\Users\Ibrahim\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
192.168.43.236 - - [05/Jun/2023 20:30:29] "GET /data HTTP/1.1" 200 -


Best parameters: {'n_estimators': 50, 'max_depth': 5}
Training score: 0.8112282193729113
Testing score: 0.8283900557147961
Mean Squared Error (MSE): 3.9879857786482686
Root Mean Squared Error (RMSE): 1.996994185932515
R-squared (R2): 0.8283900557147961
Mean Absolute Error (MAE): 1.5343133831256701

Evaluation Results:

Model: Linear Regression
best_parameters: {}
training_score: 0.6069583385419726
testing_score: 0.6302581399202787
mse: 8.592306733217784
rmse: 2.931263675143842
r2: 0.6302581399202787
mae: 2.3357423495302716
required_features: 11
input_features: 11

Model: Lasso Regression
best_parameters: {'alpha': 0.1}
training_score: 0.6071231937303293
testing_score: 0.6314032863734534
mse: 8.565695059933041
rmse: 2.9267208715443025
r2: 0.6314032863734534
mae: 2.3295890745479757
required_features: 11
input_features: 11

Model: Ridge Regression
best_parameters: {'alpha': 40}
training_score: 0.6070367560592033
testing_score: 0.6304955308087963
mse: 8.58679008619985
rmse: 2.9303225225561

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def process_data():
#     data = content["MedName"]
#     print(data)

    # Load your dataset into a pandas DataFrame
    df = pd.read_csv('DailyDataset.csv')

    # Define the date ranges for each season across all six years
    spring_ranges = [pd.date_range(start=f'{year}-03-01', end=f'{year}-05-31') for year in range(2014, 2020)]
    summer_ranges = [pd.date_range(start=f'{year}-06-01', end=f'{year}-08-31') for year in range(2014, 2020)]
    fall_ranges = [pd.date_range(start=f'{year}-09-01', end=f'{year}-11-30') for year in range(2014, 2020)]
    winter_ranges = [pd.date_range(start=f'{year}-12-01', end=f'{year + 1}-02-28') for year in range(2014, 2020)]

    # Define the temperature ranges and corresponding values for each part of the season
    cold_range = (float('-inf'), 59)
    cool_range = (59, 77)
    warm_range = (77, 95)
    hot_range = (95, float('inf'))

    df['season'] = ''

    # Loop through each row in the DataFrame
    for i, row in df.iterrows():
        # Check which season the row's date falls under
        if row['date'] in spring_ranges[0] or row['date'] in spring_ranges[1] or row['date'] in spring_ranges[2] \
                or row['date'] in spring_ranges[3] or row['date'] in spring_ranges[4] or row['date'] in spring_ranges[5]:
            season = 'spring'
        elif row['date'] in summer_ranges[0] or row['date'] in summer_ranges[1] or row['date'] in summer_ranges[2] \
                or row['date'] in summer_ranges[3] or row['date'] in summer_ranges[4] or row['date'] in summer_ranges[5]:
            season = 'summer'
        elif row['date'] in fall_ranges[0] or row['date'] in fall_ranges[1] or row['date'] in fall_ranges[2] \
                or row['date'] in fall_ranges[3] or row['date'] in fall_ranges[4] or row['date'] in fall_ranges[5]:
            season = 'fall'
        else:
            season = 'winter'
        df.at[i, 'season'] = season

        # Determine which part of the season the row's temperature falls under
        temp = row['Temperature']
        if temp < cold_range[1]:
            part = 'early'
        elif cool_range[0] <= temp < cool_range[1]:
            part = 'mid'
        elif warm_range[0] <= temp < warm_range[1]:
            part = 'late'
        else:
            part = 'end'

        # Assign the appropriate value to the row based on the part and season
        if part == 'early':
            if season == 'spring':
                df.loc[i, 'R06'] = row['R06'] + 5
            elif season == 'summer':
                df.loc[i, 'R06'] = row['R06'] + 8
            elif season == 'fall':
                df.loc[i, 'R06'] = row['R06'] + 11
            elif season == 'winter':
                df.loc[i, 'R06'] = row['R06']
        elif part == 'mid':
            if season == 'spring':
                df.loc[i, 'R06'] = row['R06'] + 5
            elif season == 'summer':
                df.loc[i, 'R06'] = row['R06'] + 8
            elif season == 'fall':
                df.loc[i, 'R06'] = row['R06'] + 11
            elif season == 'winter':
                df.loc[i, 'R06'] = row['R06']
        elif part == 'late':
            if season == 'spring':
                df.loc[i, 'R06'] = row['R06'] + 5
            elif season == 'summer':
                df.loc[i, 'R06'] = row['R06'] + 8
            elif season == 'fall':
                df.loc[i, 'R06'] = row['R06'] + 11
            elif season == 'winter':
                df.loc[i, 'R06'] = row['R06']
        else:
            if season == 'spring':
                df.loc[i, 'R06'] = row['R06'] + 1
            elif season == 'summer':
                df.loc[i, 'R06'] = row['R06'] + 2
            elif season == 'fall':
                df.loc[i, 'R06'] = row['R06'] - 2
            elif season == 'winter':
                df.loc[i, 'R06'] = row['R06']

    # Define X and y
    X = df[['Year', 'Month', 'Day', 'Temperature', 'Dew Point', 'Humidity', 'Wind', 'Wind Speed', 'Pressure',
            'IntCondition', 'Event']]
    y = df['R06']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Create a dictionary of models to test with their corresponding hyperparameters
    models = {
        'Linear Regression': {
            'model': LinearRegression(),
            'params': {}
        },
        'Lasso Regression': {
            'model': Lasso(),
            'params': {'alpha': [0.1, 1, 10, 15, 20, 40]}
        },
        'Ridge Regression': {
            'model': Ridge(),
            'params': {'alpha': [0.1, 1, 10, 15, 20, 40]}
        },
        'Decision Tree': {
            'model': DecisionTreeRegressor(),
            'params': {'max_depth': [5, 10, 15, 20, 25, 30, 40, 60, 100, 150]}
        },
         'Random Forest': {
             'model': RandomForestRegressor(),
             'params': {'n_estimators': [50], 'max_depth': [5]}
         },
        'Gradient Boosting': {
            'model': GradientBoostingRegressor(),
            'params': {'n_estimators': [50], 'max_depth': [ 5]}
        }
    }
    evaluation_results = {}  # Dictionary to store evaluation results for each model

    for model_name, model_params in models.items():
        print(f'Testing {model_name}...')
        grid_search = RandomizedSearchCV(model_params['model'], model_params['params'], cv=5, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)

        # Calculate evaluation metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        residuals = y_test - y_pred
        required_features = len(X_train.columns)  # Use the number of input features as the required features count
        input_features = X_train.columns.tolist()

        # Store the evaluation results in the dictionary
        evaluation_results[model_name] = {
            'best_parameters': grid_search.best_params_,
            'training_score': grid_search.best_score_,
            'testing_score': grid_search.score(X_test, y_test),
            'mse': mse,
            'rmse': rmse,
            'r2': r2,
            'mae': mae,
            'required_features': required_features,
            'input_features': len(input_features)
        }

        # Print the evaluation results
        print(f'Best parameters: {grid_search.best_params_}')
        print(f'Training score: {grid_search.best_score_}')
        print(f'Testing score: {grid_search.score(X_test, y_test)}')
        print(f'Mean Squared Error (MSE): {mse}')
        print(f'Root Mean Squared Error (RMSE): {rmse}')
        print(f'R-squared (R2): {r2}')
        print(f'Mean Absolute Error (MAE): {mae}')


    # Print the evaluation results dictionary
    print("\nEvaluation Results:")
    for model_name, results in evaluation_results.items():
        print(f"\nModel: {model_name}")
        for key, value in results.items():
            print(f"{key}: {value}")
            
    return jsonify(evaluation_results)
